In [26]:
from getpass import getpass
from mysql.connector import connect, Error
import pandas as pd

In [27]:
try:
    connection = connect(
        host="localhost",
        user=input("Имя пользователя: "),
        password=getpass("Пароль: "))
except Error as e:
    print(e)

Имя пользователя: root
Пароль: ········


In [28]:
cursor = connection.cursor()

# 0. Схема

![text](schema.png)

# 1. Создание базы и таблиц

Create from CRUD

In [29]:
cursor.execute('''DROP DATABASE IF EXISTS Playlist''')
connection.commit()

In [30]:
create_database = 'CREATE DATABASE IF NOT EXISTS Playlist'

create_tables = [
    '''CREATE TABLE IF NOT EXISTS Playlist.Song
    (id INT NOT NULL AUTO_INCREMENT,
    length FLOAT NOT NULL,
    rating TINYINT NULL,
    url VARCHAR(45) NOT NULL,
    name VARCHAR(50) NOT NULL,
    lyrics TEXT NULL,
    PRIMARY KEY (id),
    UNIQUE INDEX url_UNIQUE (url ASC) VISIBLE);''',
    
    '''CREATE TABLE IF NOT EXISTS Playlist.Of_Genre 
    (id_song INT NOT NULL,
    id_genre INT NOT NULL);''',
    
    '''CREATE TABLE IF NOT EXISTS Playlist.Genre 
    (id INT NOT NULL AUTO_INCREMENT,
    name VARCHAR(20) NOT NULL,
    description TEXT NULL,
    PRIMARY KEY (id),
    UNIQUE INDEX name_UNIQUE (name ASC) VISIBLE);''',
    
    '''CREATE TABLE IF NOT EXISTS Playlist.Song_By_Artist 
    (id_song INT NOT NULL,
    id_artist INT NOT NULL);''',
    
    '''CREATE TABLE IF NOT EXISTS Playlist.Artist 
    (id INT NOT NULL AUTO_INCREMENT,
    name VARCHAR(20) NOT NULL,
    description TEXT NULL,
    picture_url VARCHAR(45) NULL,
    PRIMARY KEY (id),
    UNIQUE INDEX name_UNIQUE (name ASC) VISIBLE);''',
    
    '''CREATE TABLE IF NOT EXISTS Playlist.Album (
    id INT NOT NULL AUTO_INCREMENT,
    name VARCHAR(50) NOT NULL,
    description TEXT NULL,
    picture_url VARCHAR(45) NULL,
    year SMALLINT NOT NULL,
    rating TINYINT NULL,
    PRIMARY KEY (id));''',
    
    '''CREATE TABLE IF NOT EXISTS Playlist.Album_By_Artist 
    (id_album INT NOT NULL,
    id_artist INT NOT NULL);''',
    
    '''CREATE TABLE IF NOT EXISTS Playlist.From_Album 
    (id_album INT NOT NULL,
    id_song INT NOT NULL);'''   
]

cursor.execute(create_database)
for create in create_tables:
    cursor.execute(create)
connection.commit()

# 2. Запись данных

In [31]:
insert_values = [
    (
        '''INSERT INTO Playlist.Song (length, rating, url, name, lyrics)
        VALUES (%s, %s, %s, %s, %s)''',
        (
            (286, 5, 'url-1', 'Born to Die', 'You and I - we were born to die'),
            (136, 4, 'url-2', 'Love', 'Look you kiss...'),
            (187, 5, 'url-3', 'Lust for Life', 'We dance on the H'),
            (105, 5, 'url-4', 'Rolling in the Deep', 'Theres a fire starting in my heart'),
            (142, 3, 'url-5', 'Chasing Pavements', 'Round my hometown'),
            (155, 4, 'url-6', 'Hello', 'Ive been wondering if arter all these years youd like to meet'),
            (119, 2, 'url-7', 'Каждый раз', 'Если б мне платили каждый раз'),
            (82, 3, 'url-8', 'Америка', 'Я курила тогда ментоловый'),
            (61, 5, 'url-9', 'Ночной ларёк', 'Здесь все молчаливы и все в тоске'),
            (124, 3, 'url-10', 'Люди', 'А люди любят'),
            (91, 4, 'url-11', 'Мы', 'Подожди, куда ты, молодость моя'),
            (83, 3, 'url-12', 'Вы', 'Вы наказали меня и забыли в углу'),
            (75, 2, 'url-13', 'Лампочки', 'Время сжигать мосты'),
            (62, 3, 'url-14', 'Жить в твоей голове', 'Жить в твоей голове необдуманно, нечаянно'),
            (83, 2, 'url-15', 'Ромашки', 'Привет, ромашки, платите деньги')
        )
    ),
    (
        '''INSERT INTO Playlist.Genre (name, description)
        VALUES (%s, %s)''',
        (
            ('Поп', 'простота инструментальной части, ритмичность, акцент на вокал'),
            ('Рок', 'принадлежность некоторых стилей музыки к року оспаривается'),
            ('Альтернатива', '«альтернатива» подразумевает антитезу мейнстримовой рок-музыке')
        )
    ),
    (
        '''INSERT INTO Playlist.Album (name, description, picture_url, year, rating)
        VALUES (%s, %s, %s, %s, %s)''',
        (
            ('Born to Die', 'A predominantly baroque pop and trip hop album', 'picurl-1', 2012, 4),
            ('Honeymoon', 'The album marked a departure from the more guitar-driven previous album', 'picurl-2', 2015, 5),
            ('21', 'The second studio album by the English singer-songwriter Adele', 'picurl-3', 2013, 4),
            ('Раскраски для взрослых', 'Первый студийный альбом российской певицы Монеточки', 'picurl-4', 2018, 3),
            ('Декоративно-прикладное искусство', 'Второй студийный альбом российской певицы Монеточки', 'picurl-5', 2020, 4),
            ('На вырост', 'инди-группа «Дайте танк (!)» выпустила концептуальный миньон «На вырост»', 'picurl-6', 2019, 4),
            ('Лампочки', 'Сингл Земфиры', 'picurl-7', 2017, 3),
            ('Жить в твоей голове', 'Сингл Земфиры', 'picurl-8', 2018, 4),
            ('Ромашки', 'Сингл Земфиры', 'picurl-9', 2000, 4)
        )
    ),
    (
        '''INSERT INTO Playlist.Artist (name, description, picture_url)
        VALUES (%s, %s, %s)''',
        (
            ('Lana Del Rey', 'Elizabeth Woolridge Grant is an American singer', 'picurl-1'),
            ('Adele', 'Adele Laurie Blue Adkins known mononymously as Adele, is an English singer-songwriter', 'picurl-2'),
            ('Монеточка', 'российская певица, автор песен и музыкант', 'picurl-3'),
            ('Дайте Танк (!)', 'российская рок-группа из подмосковного города Коломна', 'picurl-4'),
            ('Земфира', 'российская рок-певица, музыкант, композитор, продюсер, поэтесса и автор песен', 'picurl-5')
        )
    ),
    (
        '''INSERT INTO Playlist.Of_Genre (id_song, id_genre)
        VALUES (%s, %s)''',
        (
            (1, 1),
            (1, 3),
            (2, 1),
            (3, 3),
            (4, 1),
            (5, 1),
            (6, 1),
            (6, 3),
            (7, 1),
            (8, 3),
            (9, 3),
            (10, 2),
            (10, 3),
            (11, 2),
            (12, 2),
            (12, 3),
            (13, 3),
            (14, 3),
            (15, 2)
        )
    ),
    (
        '''INSERT INTO Playlist.Song_By_Artist (id_song, id_artist)
        VALUES (%s, %s)''',
        (
            (1, 1),
            (2, 1),
            (3, 1),
            (4, 2),
            (5, 2), 
            (6, 2),
            (7, 3),
            (8, 3),
            (9, 3),
            (10, 4),
            (11, 4),
            (12, 4),
            (13, 5),
            (14, 5),
            (15, 5)
        )
    ),
    (
        '''INSERT INTO Playlist.Album_By_Artist (id_album, id_artist)
        VALUES (%s, %s)''',
        (
            (1, 1),
            (2, 1),
            (3, 2),
            (4, 3),
            (5, 3),
            (6, 4),
            (7, 5),
            (8, 5),
            (9, 5)
        )
    ),
    (
        '''INSERT INTO Playlist.From_Album (id_song, id_album)
        VALUES (%s, %s)''',
        (
            (1, 1),
            (2, 2),
            (3, 1),
            (4, 3),
            (5, 3), 
            (6, 3),
            (7, 4),
            (8, 5),
            (9, 4),
            (10, 6),
            (11, 6),
            (12, 6),
            (13, 7),
            (14, 8),
            (15, 9)
        )
    )
]

for query, data in insert_values:
    cursor.executemany(query, data)
connection.commit()

# 3. Update

Update from CRUD: перевожу длительность трека из секунд в минуты

In [32]:
cursor.execute('SELECT * FROM Playlist.Song')
pd.DataFrame(cursor.fetchall())

,0,1,2,3,4,5
0,1,286.0,5,url-1,Born to Die,You and I - we were born to die
1,2,136.0,4,url-2,Love,Look you kiss...
2,3,187.0,5,url-3,Lust for Life,We dance on the H
3,4,105.0,5,url-4,Rolling in the Deep,Theres a fire starting in my heart
4,5,142.0,3,url-5,Chasing Pavements,Round my hometown
5,6,155.0,4,url-6,Hello,Ive been wondering if arter all these years yo...
6,7,119.0,2,url-7,Каждый раз,Если б мне платили каждый раз
7,8,82.0,3,url-8,Америка,Я курила тогда ментоловый
8,9,61.0,5,url-9,Ночной ларёк,Здесь все молчаливы и все в тоске
9,10,124.0,3,url-10,Люди,А люди любят


In [33]:
cursor.execute('''UPDATE Playlist.Song
                SET length = length / 60''')
connection.commit()

In [34]:
cursor.execute('SELECT * FROM Playlist.Song')
pd.DataFrame(cursor.fetchall())

,0,1,2,3,4,5
0,1,4.76667,5,url-1,Born to Die,You and I - we were born to die
1,2,2.26667,4,url-2,Love,Look you kiss...
2,3,3.11667,5,url-3,Lust for Life,We dance on the H
3,4,1.75000,5,url-4,Rolling in the Deep,Theres a fire starting in my heart
4,5,2.36667,3,url-5,Chasing Pavements,Round my hometown
5,6,2.58333,4,url-6,Hello,Ive been wondering if arter all these years yo...
6,7,1.98333,2,url-7,Каждый раз,Если б мне платили каждый раз
7,8,1.36667,3,url-8,Америка,Я курила тогда ментоловый
8,9,1.01667,5,url-9,Ночной ларёк,Здесь все молчаливы и все в тоске
9,10,2.06667,3,url-10,Люди,А люди любят


# 4. Select

Read from CRUD<br>
SELECT + фильтрация: выбираю альбомы, выпущенные не ранее 2018

In [35]:
cursor.execute(
    '''SELECT *
    FROM Playlist.Album
    WHERE year >= 2018
    ''')
pd.DataFrame(cursor.fetchall())

,0,1,2,3,4,5
0,4,Раскраски для взрослых,Первый студийный альбом российской певицы Моне...,picurl-4,2018,3
1,5,Декоративно-прикладное искусство,Второй студийный альбом российской певицы Моне...,picurl-5,2020,4
2,6,На вырост,инди-группа «Дайте танк (!)» выпустила концепт...,picurl-6,2019,4
3,8,Жить в твоей голове,Сингл Земфиры,picurl-8,2018,4


SELECT + группировка и аггрегация: нахожу средний рейтинг песен для исполнителей

In [36]:
cursor.execute(
    '''SELECT PLaylist.Artist.name, AVG(PLaylist.Song.rating)
    FROM Playlist.Song
    INNER JOIN Playlist.Song_By_Artist
    ON Playlist.Song.id = Playlist.Song_By_Artist.id_song
    INNER JOIN Playlist.Artist
    ON Playlist.Song_By_Artist.id_artist = Playlist.Artist.id
    GROUP BY Playlist.Artist.name
    ''')
pd.DataFrame(cursor.fetchall())

,0,1
0,Lana Del Rey,4.6667
1,Adele,4.0000
2,Монеточка,3.3333
3,Дайте Танк (!),3.3333
4,Земфира,2.3333


SELECT + вложенный запрос: нахожу исполнителей, у которых средний рейтинг по альбомам между 3 и 3.7

In [37]:
cursor.execute(
    '''SELECT *
    FROM (SELECT PLaylist.Artist.name, AVG(PLaylist.Album.rating) as rating
        FROM Playlist.Album
        INNER JOIN Playlist.Album_By_Artist
        ON Playlist.Album.id = Playlist.Album_By_Artist.id_album
        INNER JOIN Playlist.Artist
        ON Playlist.Album_By_Artist.id_artist = Playlist.Artist.id
        GROUP BY Playlist.Artist.name) as Artist_Rating
    WHERE Artist_Rating.rating BETWEEN 3 and 3.7
    ''')
out = cursor.fetchall()
pd.DataFrame(out)

,0,1
0,Монеточка,3.5000
1,Земфира,3.6667


SELECT + JOIN + вложенные запросы + фильтрация + группировка и аггрегация + сортировка: вывожу альбом с самой большой средней продолжительностью песни, не считая альтернативных песен

In [40]:
cursor.execute(
    '''
    SELECT Album_Mean.name
    FROM (SELECT Playlist.Album.name, AVG(Song_Genre.length) as mean_length
        FROM (SELECT Playlist.Song.id, Playlist.Song.length, JSON_ARRAYAGG(Playlist.Genre.name) AS genres
            FROM PLaylist.Song
            INNER JOIN Playlist.Of_Genre
            ON Playlist.Song.id = Playlist.Of_Genre.id_song
            INNER JOIN Playlist.Genre
            ON Playlist.Of_Genre.id_genre = Playlist.Genre.id
            GROUP BY Song.id, Song.length) as Song_Genre

        INNER JOIN Playlist.From_Album
        ON Song_Genre.id = Playlist.From_Album.id_song
        INNER JOIN Playlist.Album
        ON Playlist.From_Album.id_album = Playlist.Album.id

        WHERE NOT JSON_CONTAINS(Song_Genre.genres, JSON_ARRAY('Альтернатива'))
        GROUP BY Playlist.Album.id, Playlist.Album.name) as Album_Mean
    ORDER BY Album_Mean.mean_length DESC
    LIMIT 1
    ''')
pd.DataFrame(cursor.fetchall())

,0
0,Honeymoon


# 5. Процедура

Процедура: выводит продолжительность песен указанного жанра в альбомах

In [14]:
cursor.execute(
    '''
    CREATE PROCEDURE Playlist.album_time
    (genre_value VARCHAR(20))
    BEGIN
        SELECT Playlist.Album.name, SUM(Song_Genre.length) as sum_length
        FROM (SELECT Playlist.Song.id, Playlist.Song.length, JSON_ARRAYAGG(Playlist.Genre.name) AS genres
            FROM PLaylist.Song
            LEFT JOIN Playlist.Of_Genre
            ON Playlist.Song.id = Playlist.Of_Genre.id_song
            LEFT JOIN Playlist.Genre
            ON Playlist.Of_Genre.id_genre = Playlist.Genre.id
            GROUP BY Song.id, Song.length) as Song_Genre

        INNER JOIN Playlist.From_Album
        ON Song_Genre.id = Playlist.From_Album.id_song
        INNER JOIN Playlist.Album
        ON Playlist.From_Album.id_album = Playlist.Album.id

        WHERE JSON_CONTAINS(Song_Genre.genres, JSON_ARRAY(genre_value))
        GROUP BY Playlist.Album.id, Playlist.Album.name;
    END''')
connection.commit()

In [15]:
cursor.execute('CALL Playlist.album_time("Рок")')
out = cursor.fetchall()
connection.next_result()
pd.DataFrame(out)

,0,1
0,На вырост,4.966667
1,Ромашки,1.383333


In [16]:
cursor.execute('CALL Playlist.album_time("Поп")')
out = cursor.fetchall()
connection.next_result()
pd.DataFrame(out)

,0,1
0,Born to Die,4.766667
1,Honeymoon,2.266667
2,21,6.700000
3,Раскраски для взрослых,1.983333


In [17]:
cursor.execute('CALL Playlist.album_time("Альтернатива")')
out = cursor.fetchall()
connection.next_result()
pd.DataFrame(out)

,0,1
0,Born to Die,7.883333
1,21,2.583333
2,Декоративно-прикладное искусство,1.366667
3,Раскраски для взрослых,1.016667
4,На вырост,3.450000
5,Лампочки,1.250000
6,Жить в твоей голове,1.033333


# 6. Триггер

Триггер: при удалении песни из таблицы Song удаляет её вхождения в таблицу Song_By_Artist

In [18]:
cursor.execute('''CREATE TRIGGER Playlist.del_song AFTER DELETE ON Playlist.Song
                FOR EACH ROW DELETE FROM Playlist.Song_By_Artist WHERE id_song = OLD.id''')
connection.commit()

In [19]:
cursor.execute('SELECT * FROM Playlist.Song_By_Artist')
pd.DataFrame(cursor.fetchall())

,0,1
0,1,1
1,2,1
2,3,1
3,4,2
4,5,2
5,6,2
6,7,3
7,8,3
8,9,3
9,10,4


In [20]:
cursor.execute('DELETE FROM Playlist.Song WHERE length < 2')
connection.commit()
cursor.execute('SELECT * FROM Playlist.Song_By_Artist')
pd.DataFrame(cursor.fetchall())

,0,1
0,1,1
1,2,1
2,3,1
3,5,2
4,6,2
5,10,4


# 7. Временная таблица

Delete from CRUD<br>
Транзакция + временная таблица (практическая вещь): одновременно создаём временную таблицу с информацией из таблицы Genre и удаляем саму таблицу Genre

In [21]:
cursor.execute('''START TRANSACTION;
                CREATE TEMPORARY TABLE Playlist.Saved_Genre
                SELECT * FROM Playlist.Genre;
                DROP TABLE Playlist.Genre;
                COMMIT''')
while connection.next_result():
    pass
connection.commit()

In [22]:
cursor.execute('SELECT * FROM Playlist.Saved_Genre')
cursor.fetchall()

[(1, 'Поп', 'простота инструментальной части, ритмичность, акцент на вокал'),
 (2, 'Рок', 'принадлежность некоторых стилей музыки к року оспаривается'),
 (3,
  'Альтернатива',
  '«альтернатива» подразумевает антитезу мейнстримовой рок-музыке')]

In [23]:
cursor.execute('SHOW TABLES IN Playlist')
cursor.fetchall()

[('album',),
 ('album_by_artist',),
 ('artist',),
 ('from_album',),
 ('of_genre',),
 ('song',),
 ('song_by_artist',)]

# 8. Закрываем соединение

In [41]:
cursor.close()

True

In [42]:
connection.close()